In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import silhouette_score
from sklearn.svm import OneClassSVM
from lightgbm import LGBMClassifier


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten


In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

df_meta = pd.read_csv("meta_features_database.csv")

df_specifications = pd.read_excel("Resource_Specifications.xlsx")



In [55]:
random_meta = df_meta.sample(n=1)

random_meta

,n_samples,n_features,pca_expl_ratio_1,pca_expl_ratio_2,pca_expl_ratio_3,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,cor.mean,...,sd.sd,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd,Filename
58,219735,116,1.640000e+14,0.568142,1.420000e+15,7.570000e+31,334.958745,6.816298,0.553049,0.547766,...,16835.55053,2.530000e+16,1.200000e+17,1.640000e+14,1.110000e+15,1715.656926,11363.84582,-13891.41696,52536.95525,NBaIoT_device_4_4.csv


In [56]:
filename = random_meta["Filename"]
filename

58    NBaIoT_device_4_4.csv
Name: Filename, dtype: object

In [57]:
random_meta = random_meta.drop(columns=["Filename"])
random_meta

,n_samples,n_features,pca_expl_ratio_1,pca_expl_ratio_2,pca_expl_ratio_3,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,cor.mean,...,sd.mean,sd.sd,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd
58,219735,116,1.640000e+14,0.568142,1.420000e+15,7.570000e+31,334.958745,6.816298,0.553049,0.547766,...,2432.88434,16835.55053,2.530000e+16,1.200000e+17,1.640000e+14,1.110000e+15,1715.656926,11363.84582,-13891.41696,52536.95525


In [58]:
random_resource = df_specifications.sample(n=1)
random_resource = random_resource.drop(columns=["Other Factors", "M1", "M2", "M3", "M4", "M5", "M6"])
random_resource

,Device,CPU,GPU,RAM,Memory,Power Consumption,Energy Level
0,CCTVCamera1,ARM Cortex-M4,None,256MB,512MB,3W,Low


In [59]:
new_data = pd.concat([random_meta.reset_index(drop=True), random_resource.reset_index(drop=True)], axis=1)

new_data

,n_samples,n_features,pca_expl_ratio_1,pca_expl_ratio_2,pca_expl_ratio_3,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,cor.mean,...,t_mean.sd,var.mean,var.sd,Device,CPU,GPU,RAM,Memory,Power Consumption,Energy Level
0,219735,116,1.640000e+14,0.568142,1.420000e+15,7.570000e+31,334.958745,6.816298,0.553049,0.547766,...,11363.84582,-13891.41696,52536.95525,CCTVCamera1,ARM Cortex-M4,None,256MB,512MB,3W,Low


In [60]:
# Load label encoder and preprocess new data
label_encoder = joblib.load('label_encoder.pkl')

In [61]:
# Load the min_max_values DataFrame from the joblib file
min_max_values = joblib.load('min_max_values.joblib')



# Load the one-hot encoding transformation from the pickle file
one_hot_encoder= joblib.load('one_hot_encoding_mapping.pkl')

In [62]:
# Now you can use min_max_values DataFrame in your code
min_max_values

,n_samples,n_features,pca_expl_ratio_1,pca_expl_ratio_2,pca_expl_ratio_3,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,cor.mean,...,sd.mean,sd.sd,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd
min,4220,9,2.256270e-01,0.000000,2.756642e-01,9.511136e-02,0.00000,-3.585033,0.106413,0.232843,...,1.539344e-02,5.110223e-02,9.226646e-01,2.013988e-02,2.256270e-01,2.562330e-01,1.734750e-01,3.610920e-01,-1.416029e+04,0.000000e+00
max,219736,125,5.080000e+14,1.284555,2.370000e+15,2.290000e+32,19985.97736,65.307238,0.761804,1.480601,...,5.460000e+11,4.300000e+12,2.530000e+16,1.200000e+17,5.080000e+14,3.490000e+15,3.980000e+12,3.140000e+13,1.480000e+12,1.160000e+13


In [63]:
# Perform preprocessing on numeric features
numeric_features = new_data.select_dtypes(include=[np.number])
numeric_features

,n_samples,n_features,pca_expl_ratio_1,pca_expl_ratio_2,pca_expl_ratio_3,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,cor.mean,...,sd.mean,sd.sd,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd
0,219735,116,1.640000e+14,0.568142,1.420000e+15,7.570000e+31,334.958745,6.816298,0.553049,0.547766,...,2432.88434,16835.55053,2.530000e+16,1.200000e+17,1.640000e+14,1.110000e+15,1715.656926,11363.84582,-13891.41696,52536.95525


In [64]:
numeric_features = (numeric_features - min_max_values.loc['min']) / (min_max_values.loc['max'] - min_max_values.loc['min'])


In [65]:
numeric_features 

,n_samples,n_features,pca_expl_ratio_1,pca_expl_ratio_2,pca_expl_ratio_3,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,cor.mean,...,sd.mean,sd.sd,skewness.mean,skewness.sd,sparsity.mean,sparsity.sd,t_mean.mean,t_mean.sd,var.mean,var.sd
0,0.999995,0.922414,0.322835,0.442287,0.599156,0.330568,0.01676,0.15098,0.681481,0.252391,...,4.455804e-09,3.915232e-09,1.0,1.0,0.322835,0.318052,4.310260e-10,3.618944e-10,1.816736e-10,4.529048e-09


In [66]:
# Perform one-hot encoding on categorical features
categorical_features = new_data.select_dtypes(exclude=[np.number])
categorical_features

,Device,CPU,GPU,RAM,Memory,Power Consumption,Energy Level
0,CCTVCamera1,ARM Cortex-M4,None,256MB,512MB,3W,Low


In [67]:
one_hot_encoder

Index(['Device_CCTVCamera1', 'Device_CCTVCamera2', 'Device_EdgeDevice1',
       'Device_EdgeDevice2', 'Device_EdgeDevice3', 'Device_EdgeServer1',
       'Device_EdgeServer2', 'Device_IndustrialGateway',
       'Device_IndustrialSensor1', 'Device_IndustrialSensor2',
       ...
       'Power Consumption_20W', 'Power Consumption_2W', 'Power Consumption_3W',
       'Power Consumption_50W', 'Power Consumption_5W',
       'Power Consumption_80W', 'Energy Level_High', 'Energy Level_Low',
       'Energy Level_Moderate', 'Energy Level_Very Low'],
      dtype='object', length=109)

In [68]:
# Create a DataFrame with all values as 0 and columns from one_hot_encoder
category_df = pd.DataFrame(0, index=range(1), columns=one_hot_encoder)

category_df.columns

Index(['Device_CCTVCamera1', 'Device_CCTVCamera2', 'Device_EdgeDevice1',
       'Device_EdgeDevice2', 'Device_EdgeDevice3', 'Device_EdgeServer1',
       'Device_EdgeServer2', 'Device_IndustrialGateway',
       'Device_IndustrialSensor1', 'Device_IndustrialSensor2',
       ...
       'Power Consumption_20W', 'Power Consumption_2W', 'Power Consumption_3W',
       'Power Consumption_50W', 'Power Consumption_5W',
       'Power Consumption_80W', 'Energy Level_High', 'Energy Level_Low',
       'Energy Level_Moderate', 'Energy Level_Very Low'],
      dtype='object', length=109)

In [69]:
# Update values in category_df based on the values in the categorical features
for feature in categorical_features.keys():
    # Get the value from the categorical feature
    value = categorical_features[feature].iloc[0]
    # Construct the column name in category_df
    column_name = f"{feature}_{value}"
    # Check if the column exists in category_df
    if column_name in category_df.columns:
        # Set the value to 1 in the corresponding column
        category_df.loc[0, column_name] = 1



In [70]:
category_df

,Device_CCTVCamera1,Device_CCTVCamera2,Device_EdgeDevice1,Device_EdgeDevice2,Device_EdgeDevice3,Device_EdgeServer1,Device_EdgeServer2,Device_IndustrialGateway,Device_IndustrialSensor1,Device_IndustrialSensor2,...,Power Consumption_20W,Power Consumption_2W,Power Consumption_3W,Power Consumption_50W,Power Consumption_5W,Power Consumption_80W,Energy Level_High,Energy Level_Low,Energy Level_Moderate,Energy Level_Very Low
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [71]:
df = pd.concat([numeric_features, category_df], axis=1)

In [72]:
df

,n_samples,n_features,pca_expl_ratio_1,pca_expl_ratio_2,pca_expl_ratio_3,attr_conc.mean,attr_conc.sd,attr_ent.mean,attr_ent.sd,cor.mean,...,Power Consumption_20W,Power Consumption_2W,Power Consumption_3W,Power Consumption_50W,Power Consumption_5W,Power Consumption_80W,Energy Level_High,Energy Level_Low,Energy Level_Moderate,Energy Level_Very Low
0,0.999995,0.922414,0.322835,0.442287,0.599156,0.330568,0.01676,0.15098,0.681481,0.252391,...,0,0,1,0,0,0,0,1,0,0


In [73]:

# Load the best performing model
best_model = joblib.load('best_model.pkl')
predictions = best_model.predict(df)



In [74]:
print(predictions)

[9]


In [75]:
labels = {value: key for key, value in label_encoder.items()}

In [76]:
labels = {value: key for key, value in label_encoder.items()}
# Retrieve the name corresponding to the key
algorithm_name = labels[predictions[0]]

print(f"The algorithm name corresponding to key {predictions} is: {algorithm_name}")

The algorithm name corresponding to key [9] is: RandomForest


In [77]:
print(filename.to_numpy()[0])

NBaIoT_device_4_4.csv


In [78]:
path = f"Data\\"+filename.to_numpy()[0]
path

'Data\\NBaIoT_device_4_4.csv'

In [79]:
data = pd.read_csv(path)

In [80]:
data.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,Class
0,3.399075,64.490375,6.778783e+00,3.622769,64.492151,6.773485e+00,3.948449,64.527356,6.677006,12.004924,...,-1.730000e-10,-2.760000e-06,19.219261,66.039917,0.690949,116.886484,199.624329,-0.182754,-0.018720,0
1,2.564029,66.000000,9.090000e-13,2.725369,66.000000,0.000000e+00,2.954431,66.000084,0.001687,9.645510,...,-1.860000e-08,-9.220000e-06,19.339459,66.092755,1.050931,116.903277,200.004974,-0.427671,-0.028775,0
2,4.280267,66.000001,1.285580e-04,4.545122,66.001013,9.117662e-02,4.902079,66.870976,77.629260,13.081771,...,-2.370000e-06,-1.041360e-04,19.143341,66.172438,1.428120,117.421064,186.022275,-0.593434,-0.030468,0
3,1.000000,66.000000,0.000000e+00,1.000180,66.000000,6.250000e-10,1.118986,66.003319,0.309957,10.063042,...,-1.827573e-03,-4.552697e-03,20.444434,66.213615,2.431245,114.800367,476.985543,18.643230,0.351121,0
4,1.815007,66.000000,0.000000e+00,1.884615,66.000000,1.180000e-07,2.037925,66.006817,0.082878,9.617030,...,3.810000e-14,2.900000e-09,20.204512,66.560529,10.080215,120.119268,6701.923665,3.945869,0.004782,0


In [81]:
from Model_Run import get_model_accuracy

In [82]:
accuracy = get_model_accuracy(path, algorithm_name)

RandomForestClassifier()


In [83]:
print(accuracy)

0.9999772453182242
